In [1]:
# 다음에 해볼 것
# 1. 전처리 더 하기
# 2. lemmatize 해보기
# 3. 연도별 잘라서 관측
# 4. 저자 EDA (소속, 나라)
# 5. 최적 토픽 수
# 6. 하이퍼 파라미터 바꿔보기 (알파, 베타)

In [2]:
import gensim
import pickle
import pandas as pd


In [3]:
df = pd.read_pickle("../files/final_df.pickle")

In [4]:
# load corpus, dictionary

with open('../files/lda/corpus.pickle', 'rb') as f:
    corpus = pickle.load(f) # 단 한줄씩 읽어옴

with open('../files/lda/dictionary.pickle', 'rb') as f:
    dictionary = pickle.load(f) # 단 한줄씩 읽어옴


In [5]:
model = gensim.models.ldamodel.LdaModel.load("../files/lda/lda.model")

In [7]:
topics = model.print_topics(num_words=20)

In [8]:
for t in topics:
    print(t)

# 일단 작성
not_decided_del_words = ["P.", "A.", "B."]

del_words = ["%", ">", "<", "[", "]", "P", "p", "+/-", "mm", "found", "=", "CI"]

(0, '0.021*"treatment" + 0.017*"periodontal" + 0.017*"probing" + 0.016*"clinical" + 0.015*"group" + 0.015*"patients" + 0.013*"sites" + 0.013*"therapy" + 0.012*"study" + 0.012*"mm" + 0.011*"months" + 0.011*"significant" + 0.010*"depth" + 0.010*"plaque" + 0.010*"attachment" + 0.010*"%" + 0.010*"control" + 0.010*"groups" + 0.010*"baseline" + 0.009*"pocket"')
(1, '0.029*"gingivalis" + 0.022*"P." + 0.012*"activity" + 0.010*"antibody" + 0.010*"actinomycetemcomitans" + 0.010*"B." + 0.008*"A." + 0.007*"IgG" + 0.006*"cells" + 0.006*"Bacteroides" + 0.006*"periodontitis" + 0.006*"human" + 0.006*"Porphyromonas" + 0.006*"strains" + 0.006*"antibodies" + 0.005*"protein" + 0.005*"LJP" + 0.005*"bacteria" + 0.004*"cell" + 0.004*"proteins"')
(2, '0.036*"patients" + 0.030*"periodontitis" + 0.028*"levels" + 0.018*"healthy" + 0.018*"GCF" + 0.016*"periodontal" + 0.015*"gingival" + 0.014*"subjects" + 0.011*"group" + 0.010*"fluid" + 0.010*"disease" + 0.010*"sites" + 0.009*"serum" + 0.009*"study" + 0.009*"crevi

In [ ]:
!pip install tqdm

In [9]:
from tqdm.notebook import tqdm

In [10]:
# 토픽1의 비율이 50%가 넘는 문서의 index
topic_1_list = []
check_topic = model[corpus]

for i, topic_dist in tqdm(enumerate(check_topic)):
    for idx, dist in topic_dist:
        if idx == 1 and dist >= 0.5:
            topic_1_list.append(i)

In [12]:
topic_1_list

[2,
 48,
 193,
 205,
 263,
 425,
 430,
 460,
 461,
 483,
 488,
 530,
 561,
 631,
 641,
 647,
 700,
 704,
 815,
 854,
 865,
 915,
 933,
 948,
 1039,
 1072,
 1073,
 1076,
 1098,
 1109,
 1180,
 1191,
 1232,
 1247,
 1291,
 1339,
 1365,
 1379,
 1483,
 1525,
 1540,
 1547,
 1548,
 1561,
 1576,
 1580,
 1667,
 1669,
 1674,
 1736,
 1743,
 1749,
 1765,
 1848,
 1891,
 1896,
 1928,
 1972,
 2001,
 2028,
 2032,
 2048,
 2085,
 2161,
 2317,
 2323,
 2337,
 2343,
 2392,
 2411,
 2513,
 2525,
 2531,
 2535,
 2594,
 2679,
 2732,
 2816,
 2837,
 2838,
 2905,
 2909,
 2943,
 2947,
 3012,
 3026,
 3047,
 3054,
 3076,
 3172,
 3245,
 3279,
 3287,
 3302,
 3319,
 3341,
 3353,
 3430,
 3432,
 3532,
 3557,
 3591,
 3636,
 3654,
 3672,
 3701,
 3705,
 3728,
 3736,
 3752,
 3764,
 3770,
 3805,
 3868,
 3884,
 3924,
 3930,
 3931,
 3947,
 3986,
 4110,
 4176,
 4239,
 4240,
 4332,
 4407,
 4445,
 4475,
 4508,
 4509,
 4542,
 4552,
 4591,
 4657,
 4734,
 4735,
 4749,
 4751,
 4784,
 4857,
 4867,
 4889,
 4918,
 4949,
 4950,
 4970,
 4983

In [ ]:
# for문 예시
import time

for i in tqdm(range(10000)):
    time.sleep(0.001)

In [14]:
df.iloc[topic_1_list, 2][:205].values

array(['Doxycycline hyclate-loaded in situ forming gels composed from bleached shellac,\nEthocel, and Eudragit RS for periodontal pocket delivery.',
       'Decreased temperature increases the expression of a disordered bacterial late\nembryogenesis abundant (LEA) protein that enhances natural transformation.',
       'Mammalian-like type II glutaminyl cyclases in Porphyromonas gingivalis and other \noral pathogenic bacteria as targets for treatment of periodontitis.',
       'Structural and kinetic characterization of Porphyromonas gingivalis glutaminyl\ncyclase.',
       'The association of apical periodontitis and type 2 diabetes mellitus: A large\nhospital network cross-sectional case-controlled study.',
       'Complementation in trans of Porphyromonas gingivalis Lipopolysaccharide\nBiosynthetic Mutants Demonstrates Lipopolysaccharide Exchange.',
       'Synergistic effects of D-arginine, D-methionine and D-histidine against\nPorphyromonas gingivalis biofilms.',
       'Porphyromo

In [15]:
# 연도별 잘라서 관측
df.head()


for i, topic_list in enumerate(model[corpus]):
    if i == 5:
        break
    print(topic_list)
# 각 문서당 토픽의 분포를 알고 있음
# 1. 10년 단위로 문서를 묶는다
# 2. 년도에서 토픽의 분포를 전부 더해줌
# 3. 가장 마지막에 문서의 갯수로 나눠줌


[(1, 0.1511391), (2, 0.05872507), (4, 0.03236211), (7, 0.75353473)]
[(7, 0.047075856), (8, 0.022592774), (9, 0.925027)]
[(1, 0.6905827), (3, 0.071181454), (4, 0.05214415), (6, 0.018421894), (7, 0.14275701), (9, 0.021085707)]
[(0, 0.5703773), (9, 0.4252539)]
[(3, 0.08284176), (5, 0.22719537), (6, 0.55435896), (9, 0.13041262)]


In [16]:
df.reset_index(drop=True, inplace=True)

In [19]:
# 1980년대 논문으로 테스트
test = df[(df.year >= '1980') & (df.year < '1990')]
print(len(test))
test.head()

727


,journal_name,year,title,author,author_info,abstract,pos_tag_abstract
29999,Z Stomatol,1989,[Oral manifestations of HIV infection].,[Article in German],"Puelacher W, Zangerle R, Kulmer S, Waldhart E,...",Skin and mucous membranes including the oral m...,"[Skin, mucous, membranes, including, oral, muc..."
30000,Zh Mikrobiol Epidemiol Immunobiol,1989,[The tentative identification of actinomycetes...,[Article in Russian],"Mel'nikov VG, Oleĭnik II.",In this work the results obtained in the study...,"[work, results, obtained, study, morphology, A..."
30001,Aust Dent J,1989,Differential diagnosis of severe periodontal l...,"Hirsch RS, Clarke NG.",Comment in\n Aust Dent J. 1990 Apr;35(2):19...,"In current clinical practice, a differential d...","[current, clinical, practice, differential, di..."
30002,Scand J Dent Res,1989,"Gingival fluid, beta 2-microglobulin and prote...","Syrjänen SM(1), Alakuijala P, Markkanen SO, Ma...","Author information: \n(1)Faculty of Dentistry,...","beta 2-microglobulin (beta 2-m), lysozyme and ...","[beta, lysozyme, protein, concentrations, ging..."
30003,J Periodontol,1989,A comparison of freeze-dried bone allograft an...,"Rummelhart JM(1), Mellonig JT, Gray JL, Towle HJ.","Author information: \n(1)Naval Dental Clinic, ...",This study was conducted to clinically compare...,"[study, conducted, compare, freeze-dried, bone..."


In [ ]:
# 1980년대 논문만 corpus에서 가져옴

index_1980 = list(test.index)
# corpus_1980 = corpus[0,1,2,3]
corpus_1980 = corpus[29999:30726]

In [ ]:
dist_dict = {i:0 for i, value in enumerate(range(10))}

# dist_dict
for topic_dist in model[corpus_1980]:
    for key, value in topic_dist:
        dist_dict[key] += value

In [ ]:
for key, value in dist_dict.items():
    dist_dict[key] = dist_dict[key] / len(corpus_1980)

In [ ]:
dist_dict

In [ ]:
# 합이 1에 가깝게 잘 나옴
sum(dist_dict.values())